In [1]:
from mp_api.client import MPRester

key = "qx3M9lC8cMaGU3gp0ZpKSPor69SxkCU9"

c:\Users\victo\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
import plotly.graph_objects as go
import pandas as pd
import time
from numpy import mean
from datetime import date

import sqlite3
import os

### Utilizando API para coletar dados de todos os materiais

In [3]:
with MPRester(key) as mpr:
    # material = "mp-20138"
    # dossi = mpr.get_dos_by_material_id("mp-149")
    # dos = mpr.get_dos_by_material_id(material)
    # fp = dos.get_dos_fp()

    ms = mpr.summary.search(elements=['P', 'Cl'])

c:\Users\vsavanini\AppData\Local\Programs\Python\Python310\lib\site-packages\mp_api\client\mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
C:\Users\vsavanini\AppData\Local\Temp\ipykernel_25088\3178631355.py:7: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  ms = mpr.summary.search(elements=['P', 'Cl'])
Retrieving SummaryDoc documents: 100%|██████████| 309/309 [00:00<?, ?it/s]


### Funções para encontrar as densidades

In [4]:
def get_sites(mpe):
    dfs = []
    for coords in mpe.structure.sites:
        fracs = list(coords.__dict__["_frac_coords"])
        dfs.append(
            pd.DataFrame(
                {
                    "element": [str(coords.specie)],
                    "x": [fracs[0]],
                    "y": [fracs[1]],
                    "z": [fracs[2]],
                }
            )
        )
    return pd.concat(dfs, ignore_index=True).to_json(orient='index')

In [5]:
def find_zero(df):
    def interpol(dfi: pd.DataFrame):
        dfi.reset_index(drop=True, inplace=True)
        return -(
            dfi.energies[0] * dfi.densities[1] - dfi.energies[1] * dfi.densities[0]
        ) / (dfi.energies[1] - dfi.energies[0])

    serie = (
        df.reset_index(drop=True).energies.shift(1) * df.reset_index(drop=True).energies
    )
    turn_point = serie[serie < 0].index[0]

    return interpol(df[turn_point - 1 : turn_point + 1])

In [19]:
# Encontrar DOS na Efermi para cada elemento
def get_dos_at_efermi(material_id: str) -> pd.DataFrame:
    "Encontra "
    dos = mpr.get_dos_by_material_id(material_id)
    ed = dos.get_element_dos()

    element_values = {}
    for element in ed:
        e_dos = ed[element]
        e_df = pd.DataFrame(
            {
                "energies": e_dos.energies - e_dos.efermi,
                "densities": e_dos.get_densities(),
            }
        )
        element_values[str(element)] = find_zero(e_df)

    dg = pd.DataFrame(
        {"energies": dos.energies - dos.efermi, "densities": dos.get_densities()}
    )

    return find_zero(dg), str(element_values)

In [17]:
def to_sql_list(itens: list) -> str:
    "Converte uma lista Python em uma lista formatada para consultas SQL."

    def _convert(x):
        if isinstance(x, str):
            x = x.replace("'", "''")
            return f"'{x}'"
        if (pd.isna(x)) or (x=='None'):
            return "NULL"
        elif isinstance(x, date):
            return f"'{x}'"
        else:
            return str(x)

    return f'({",".join(map(_convert, itens))})'


### Coletando dados

In [20]:
db_file = "mp_database.db"


with sqlite3.connect(db_file) as conn:
    print('Cenection created')
    conn.executescript("delete from materials")

    t1 = time.time()
    for mms in ms:
        li = []
        
        if time.time() > (t1 + 60):
            break
        try:
            all_dos = get_dos_at_efermi(str(mms.material_id))
        except:
            all_dos = [None, None]

        li.append(
            [
                int(str(mms.material_id)[3:]),
                mms.nelements,
                mms.nsites,
                str(mms.composition),
                mms.formula_pretty,
                mms.volume,
                mms.density,
                mms.density_atomic,
                str(mms.symmetry.crystal_system),
                mms.symmetry.symbol,
                mms.symmetry.number,
                str(mms.material_id),
                mms.is_stable,
                mms.is_magnetic,
                mms.is_metal,
                mms.is_gap_direct,
                mms.energy_per_atom,
                mms.efermi,
                mms.total_magnetization,
                mms.last_updated,
                mms.deprecated,
                str({
                    "abc": list(mms.structure.lattice.abc),
                    "angles": list(mms.structure.lattice.angles),
                }),
                str(get_sites(mms)),
                all_dos[0],
                all_dos[1],
            ]
        )
        
        conn.executescript(
            f"""
                insert into materials (
                    id,
                    n_elements,
                    'n_atoms',
                    composition,
                    formula,
                    volume,
                    density,
                    atomic_density,
                    symetry,
                    symetry_symbol,
                    symetry_number,
                    material_id,
                    is_stable,
                    is_magnetic,
                    is_metal,
                    is_gap_direct,
                    energy_per_atom,
                    efermi,
                    total_magnetization,
                    last_updated,
                    deprecated,
                    lattice_structure,
                    element_coords,
                    dos_at_efermi,
                    elements_dos_at_efermi
                )
                values
                {to_sql_list(li[0])}
                """
        )

all_mp = (
    pd.DataFrame(
        li,
        columns=[
            "id",
            "n_elements",
            'n_atoms',
            "composition",
            "formula",
            "volume",
            "density",
            "atomic_density",
            "symetry",
            "symetry_symbol",
            "symetry_number",
            "material_id",
            "is_stable",
            "is_magnetic",
            "is_metal",
            "is_gap_direct",
            "energy_per_atom",
            "efermi",
            "total_magnetization",
            "last_updated",
            "deprecated",
            "lattice_structure",
            "element_coords",
            "dos_at_efermi",
            "elements_dos_at_efermi",
        ],
    )
    .sort_values("id")
    .set_index("id")
)

Cenection created


Retrieving ElectronicStructureDoc documents: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [ ]:

with sqlite3.connect(db_file) as conn:
    

In [32]:
all2 = all_mp[
    [
        "material_id",
        "formula",
        "n_elements",
        "composition",
        "volume",
        "density",
        "atomic_density",
        "symetry",
        "symetry_symbol",
        "symetry_number",
        "is_stable",
        "is_magnetic",
        "is_metal",
        "is_gap_direct",
        "energy_per_atom",
        "efermi",
        "total_magnetization",
        "lattice_structure",
        "element_coords",
        "dos_at_efermi",
        "elements_dos_at_efermi",
        "last_updated",
        "deprecated",
    ]
]

### Gráficos

In [ ]:
dg = pd.DataFrame(
    {"energies": dos.energies - dos.efermi, "densities": dos.get_densities()}
)

In [ ]:
fdos = dg.query(f"energies<{dos.efermi/2} & energies>{-dos.efermi/2}")

In [ ]:
go.Figure(
    [
        # go.Scatter(x=data.energies, y=data.densities*20),
        go.Scatter(x=fdos.energies, y=fdos.densities),
    ]
    + plots
)

### Criando banco de dados

In [28]:
all_mp

,n_elements,n_atoms,composition,formula,volume,density,atomic_density,symetry,symetry_symbol,symetry_number,...,is_gap_direct,energy_per_atom,efermi,total_magnetization,last_updated,deprecated,lattice_structure,element_coords,dos_at_efermi,elements_dos_at_efermi
id,,,,,,,,,,,,,,,,,,,,,
23230,2,16,P4 Cl12,PCl3,484.509915,1.882698,30.281870,Orthorhombic,Pnma,62,...,False,-3.901169,-1.614793,0.000000,2022-10-28 00:54:36.940,False,"{'abc': [6.22185524, 8.17878846, 9.5212461], '...","{""0"":{""element"":""P"",""x"":0.92623871,""y"":0.98065...",NaN,None
27788,3,24,Ta2 P2 Cl20,TaPCl10,672.079793,2.799117,28.003325,Triclinic,P-1,2,...,False,-4.498788,-0.759506,0.000000,2022-10-28 00:53:50.868,False,"{'abc': [7.009559334969267, 7.346487315117377,...","{""0"":{""element"":""Ta"",""x"":0.0,""y"":0.0,""z"":0.5},...",NaN,None
28387,3,40,P4 Au4 Cl32,PAuCl8,1162.707212,2.922395,29.067680,Orthorhombic,Imma,74,...,False,-3.357039,-0.458528,0.000000,2022-10-28 00:54:28.178,False,"{'abc': [11.672381289798967, 11.67263751426426...","{""0"":{""element"":""P"",""x"":0.2532266,""y"":0.253234...",NaN,None
28426,3,80,B32 P16 Cl32,B2PCl2,2145.729149,1.529211,26.821614,Orthorhombic,Pbcn,60,...,False,-5.217447,-1.109159,0.000000,2022-10-28 00:53:46.719,False,"{'abc': [6.510499, 13.271094, 24.834413], 'ang...","{""0"":{""element"":""B"",""x"":0.610346,""y"":0.858407,...",NaN,None
29592,3,22,Re2 P2 Cl18,RePCl9,589.988394,3.018625,26.817654,Triclinic,P-1,2,...,False,-4.255946,1.807285,6.000000,2022-10-28 00:54:30.648,False,"{'abc': [6.998466733283951, 9.123501776911715,...","{""0"":{""element"":""Re"",""x"":0.61173659,""y"":0.6319...",0.661775,"{'Re': 0.36894192045454627, 'P': 0.00150017045..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217036,4,36,U4 P4 Cl4 O24,UPClO6,671.318504,3.962151,18.647736,Tetragonal,I-4,82,...,False,-7.693654,-0.403265,8.002130,2022-10-28 00:53:52.141,False,"{'abc': [10.417299074967465, 10.41729907496746...","{""0"":{""element"":""U"",""x"":0.366223,""y"":0.172271,...",NaN,None
1218945,5,84,Sr20 P12 Br3 Cl1 O48,Sr20P12Br3ClO48,1282.571458,4.100586,15.268708,Triclinic,P-1,2,...,False,-7.387495,1.627473,0.000011,2022-10-28 00:53:58.197,False,"{'abc': [10.106102, 12.437613195011693, 12.437...","{""0"":{""element"":""Sr"",""x"":0.66687,""y"":0.582379,...",NaN,None
1225035,7,148,La2 P4 H88 C28 N14 Cl8 O4,LaP2H44C14N7(Cl2O)2,1612.983029,1.410824,10.898534,Monoclinic,Cc,9,...,True,-5.469116,-0.248650,0.056176,2022-10-28 00:53:50.905,False,"{'abc': [10.582962998682834, 10.58296299868283...","{""0"":{""element"":""La"",""x"":0.256109,""y"":0.749017...",NaN,None


In [24]:
schema = """
-- drop table temp_materials;
create table temp_materials (
    id int primary key,
    material_id text not null,
    formula text,
    n_elements int,
    n_atoms int,
    composition text,
    volume real,
    density real,
    atomic_density real,
    symetry text,
    symetry_symbol text,
    symetry_number int,
    is_stable boolean,
    is_magnetic boolean,
    is_metal boolean,
    is_gap_direct boolean,
    energy_per_atom real,
    efermi real,
    total_magnetization real,
    lattice_structure text,
    element_coords text,
    dos_at_efermi real,
    elements_dos_at_efermi text,
    last_updated datetime,
    deprecated boolean
)
"""

In [3]:

def check_db(filename):
    return os.path.exists(filename)


db_file = "mp_database.db"

# if check_db(db_file):
#     print("Database already exists. Exiting...")
#     exit(0)


In [4]:
with sqlite3.connect(db_file) as conn:
    print("Created the connection!")
    # Execute the SQL query to create the table
    conn.executescript(schema)
    print("Created the Table!")
    # conn.executescript(
    #     f"""
    #         insert into materials (name, size, date)
    #         values
    #         {txt}
    #         """
    # )

Created the connection!


NameError: name 'schema' is not defined

In [5]:

with sqlite3.connect(db_file) as conn:
    
            already_in_temp = [x[0] for x in conn.cursor().execute('select id from materials')]

In [6]:
len(already_in_temp)

155739

In [2]:
for row in conn.cursor().execute('select count(*) from temp_materials'):
    print(row)

NameError: name 'conn' is not defined